# M2177.003100 Deep Learning <br> Assignment #3 Part 5: Transformer

Copyright (C) Data Science Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Jeonghee Jo, October 2019

This is about Transformer (Vaswani et al., 2017).
https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf

Original blog post & code:
https://github.com/Kyubyong/transformer
##### Copyright 2019 The TensorFlow Authors.

That said, you are allowed to copy paste the codes from the original repo.
HOWEVER, <font color=red> try to implement the model yourself first </font>, and consider the original source code as a last resort.
You will learn a lot while wrapping around your head during the implementation. And you will understand nuts and bolts of RNNs more clearly in a code level.

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.  
Once you have done **all Assignment Part 1-5**, run the *CollectSubmission.sh* script with your **Team number** as input argument. <br>
This will produce a zipped file called *[Your team number].zip*. Please submit this file on ETL. &nbsp;&nbsp; (Usage: ./*CollectSubmission.sh* team_#)

### Transformer (20 points)

This assignment is an on/off one: just make this notebook **"work"** without problem by: 

1. **Explore various hyperparameters and pick the best set (in class Hparams, transformer_modules.py)** 

### The Grading is as follows:

1. Train your model using at least <font color=red> 12 different hyperparameter set </font>. Report performance results (BLEU score) on given test set <font color=red> for corresponding each hyperparameter set </font>. 

2. Plus, <font color=red> submit the one checkpoint file </font> of your best model. 

The details are described in <font color=red>**transformer_modules.py**</font>. (There is nothing to implement in this notebook.)


Now proceed to the code.


In [4]:
from transformer_utils import *
from transformer_modules import *

In [7]:
from tqdm import tqdm
import os
import argparse
import math
import os
import errno
import sentencepiece as spm
import re
import logging

# Preprocessing

In [8]:
def prepro(hp):
    """Load raw data -> Preprocessing -> Segmenting with sentencepice
    hp: hyperparams. argparse.
    """
    
    train1 = "./iwslt2016/de-en/train.tags.de-en.de"
    train2 = "./iwslt2016/de-en/train.tags.de-en.en"
    eval1 = "./iwslt2016/de-en/IWSLT16.TED.tst2013.de-en.de.xml"
    eval2 = "./iwslt2016/de-en/IWSLT16.TED.tst2013.de-en.en.xml"
    test1 = "./iwslt2016/de-en/IWSLT16.TED.tst2014.de-en.de.xml"
    test2 = "./iwslt2016/de-en/IWSLT16.TED.tst2014.de-en.en.xml"
    for f in (train1, train2, eval1, eval2, test1, test2):
        if not os.path.isfile(f):
            raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), f)

    # train
    _prepro = lambda x:  [line.strip() for line in open(x, 'r').read().split("\n") \
                      if not line.startswith("<")]
    prepro_train1, prepro_train2 = _prepro(train1), _prepro(train2)
    assert len(prepro_train1)==len(prepro_train2), "Check if train source and target files match."

    # eval
    _prepro = lambda x: [re.sub("<[^>]+>", "", line).strip() \
                     for line in open(x, 'r').read().split("\n") \
                     if line.startswith("<seg id")]
    prepro_eval1, prepro_eval2 = _prepro(eval1), _prepro(eval2)
    assert len(prepro_eval1) == len(prepro_eval2), "Check if eval source and target files match."

    # test
    prepro_test1, prepro_test2 = _prepro(test1), _prepro(test2)
    assert len(prepro_test1) == len(prepro_test2), "Check if test source and target files match."

    os.makedirs("./iwslt2016/prepro", exist_ok=True)
    def _write(sents, fname):
        with open(fname, 'w') as fout:
            fout.write("\n".join(sents))

    _write(prepro_train1, "./iwslt2016/prepro/train.de")
    _write(prepro_train2, "./iwslt2016/prepro/train.en")
    _write(prepro_train1+prepro_train2, "./iwslt2016/prepro/train")
    _write(prepro_eval1, "./iwslt2016/prepro/eval.de")
    _write(prepro_eval2, "./iwslt2016/prepro/eval.en")
    _write(prepro_test1, "./iwslt2016/prepro/test.de")
    _write(prepro_test2, "./iwslt2016/prepro/test.en")

    os.makedirs("./iwslt2016/segmented", exist_ok=True)
    train = '--input=./iwslt2016/prepro/train --pad_id=0 --unk_id=1 \
             --bos_id=2 --eos_id=3\
             --model_prefix=./iwslt2016/segmented/bpe --vocab_size={} \
             --model_type=bpe'.format(hp.vocab_size)
    #print(train)
    spm.SentencePieceTrainer.Train(train)

    sp = spm.SentencePieceProcessor()
    sp.Load("./iwslt2016/segmented/bpe.model")

    def _segment_and_write(sents, fname):
        with open(fname, "w") as fout:
            for sent in sents:
                pieces = sp.EncodeAsPieces(sent)
                fout.write(" ".join(pieces) + "\n")

    _segment_and_write(prepro_train1, "./iwslt2016/segmented/train.de.bpe")
    _segment_and_write(prepro_train2, "./iwslt2016/segmented/train.en.bpe")
    _segment_and_write(prepro_eval1, "./iwslt2016/segmented/eval.de.bpe")
    _segment_and_write(prepro_eval2, "./iwslt2016/segmented/eval.en.bpe")
    _segment_and_write(prepro_test1, "./iwslt2016/segmented/test.de.bpe")

    print("train1:", open("./iwslt2016/segmented/train.de.bpe",'r').readline())
    print("train2:", open("./iwslt2016/segmented/train.en.bpe", 'r').readline())
    print("eval1:", open("./iwslt2016/segmented/eval.de.bpe", 'r').readline())
    print("eval2:", open("./iwslt2016/segmented/eval.en.bpe", 'r').readline())
    print("test1:", open("./iwslt2016/segmented/test.de.bpe", 'r').readline())

In [11]:
hparams = Hparams()
parser = hparams.parser
hp = parser.parse_args()
hp.vocab_size=1000
prepro(hp)

train1: ▁D a v id ▁G all o : ▁Das ▁ist ▁B ill ▁L ange . ▁Ich ▁bin ▁D a ve ▁G all o .

train2: ▁D a v id ▁G all o : ▁This ▁is ▁B ill ▁L ange . ▁I ' m ▁D a ve ▁G all o .

eval1: ▁Als ▁ich ▁1 1 ▁J ahre ▁al t ▁war , ▁wurde ▁ich ▁eines ▁M or gen s ▁von ▁den ▁K l än gen ▁h ell er ▁F re u de ▁ge we ck t .

eval2: ▁Wh en ▁I ▁was ▁1 1 , ▁I ▁re m em ber ▁w ak ing ▁up ▁one ▁m or n ing ▁to ▁the ▁s ound ▁of ▁j o y ▁in ▁my ▁h ou se .

test1: ▁Als ▁ich ▁in ▁mein en ▁20 ern ▁war , ▁hatte ▁ich ▁meine ▁er ste ▁P s y ch other ap ie - P at ient in .



# Transformer

In [12]:
class Transformer:
    '''
    xs: tuple of
        x: int32 tensor. (N, T1)
        x_seqlens: int32 tensor. (N,)
        sents1: str tensor. (N,)
    ys: tuple of
        decoder_input: int32 tensor. (N, T2)
        y: int32 tensor. (N, T2)
        y_seqlen: int32 tensor. (N, )
        sents2: str tensor. (N,)
    training: boolean.
    '''
    def __init__(self, hp):
        self.hp = hp
        self.token2idx, self.idx2token = load_vocab(hp.vocab)
        self.embeddings = get_token_embeddings(self.hp.vocab_size, self.hp.d_model, zero_pad=True)

    def encode(self, xs, training=True):
        '''
        Returns
        memory: encoder outputs. (N, T1, d_model)
        '''
        with tf.variable_scope("encoder", reuse=tf.AUTO_REUSE):
            x, seqlens, sents1 = xs

            # src_masks
            src_masks = tf.math.equal(x, 0) # (N, T1)

            # embedding
            enc = tf.nn.embedding_lookup(self.embeddings, x) # (N, T1, d_model)
            enc *= self.hp.d_model**0.5 # scale

            enc += positional_encoding(enc, self.hp.maxlen1)
            enc = tf.layers.dropout(enc, self.hp.dropout_rate, training=training)

            ## Blocks
            for i in range(self.hp.num_blocks):
                with tf.variable_scope("num_blocks_{}".format(i), reuse=tf.AUTO_REUSE):
                    # self-attention
                    enc = multihead_attention(queries=enc,
                                              keys=enc,
                                              values=enc,
                                              key_masks=src_masks,
                                              num_heads=self.hp.num_heads,
                                              dropout_rate=self.hp.dropout_rate,
                                              training=training,
                                              causality=False)
                    # feed forward
                    enc = ff(enc, num_units=[self.hp.d_ff, self.hp.d_model])
        memory = enc
        return memory, sents1, src_masks

    def decode(self, ys, memory, src_masks, training=True):
        '''
        memory: encoder outputs. (N, T1, d_model)
        src_masks: (N, T1)
        Returns
        logits: (N, T2, V). float32.
        y_hat: (N, T2). int32
        y: (N, T2). int32
        sents2: (N,). string.
        '''
        with tf.variable_scope("decoder", reuse=tf.AUTO_REUSE):
            decoder_inputs, y, seqlens, sents2 = ys

            # tgt_masks
            tgt_masks = tf.math.equal(decoder_inputs, 0)  # (N, T2)

            # embedding
            dec = tf.nn.embedding_lookup(self.embeddings, decoder_inputs)  # (N, T2, d_model)
            dec *= self.hp.d_model ** 0.5  # scale

            dec += positional_encoding(dec, self.hp.maxlen2)
            dec = tf.layers.dropout(dec, self.hp.dropout_rate, training=training)

            # Blocks
            for i in range(self.hp.num_blocks):
                with tf.variable_scope("num_blocks_{}".format(i), reuse=tf.AUTO_REUSE):
                    # Masked self-attention (Note that causality is True at this time)
                    dec = multihead_attention(queries=dec,
                                              keys=dec,
                                              values=dec,
                                              key_masks=tgt_masks,
                                              num_heads=self.hp.num_heads,
                                              dropout_rate=self.hp.dropout_rate,
                                              training=training,
                                              causality=True,
                                              scope="self_attention")

                    # Vanilla attention
                    dec = multihead_attention(queries=dec,
                                              keys=memory,
                                              values=memory,
                                              key_masks=src_masks,
                                              num_heads=self.hp.num_heads,
                                              dropout_rate=self.hp.dropout_rate,
                                              training=training,
                                              causality=False,
                                              scope="vanilla_attention")
                    ### Feed Forward
                    dec = ff(dec, num_units=[self.hp.d_ff, self.hp.d_model])

        # Final linear projection (embedding weights are shared)
        weights = tf.transpose(self.embeddings) # (d_model, vocab_size)
        logits = tf.einsum('ntd,dk->ntk', dec, weights) # (N, T2, vocab_size)
        y_hat = tf.to_int32(tf.argmax(logits, axis=-1))

        return logits, y_hat, y, sents2

    def train(self, xs, ys):
        '''
        Returns
        loss: scalar.
        train_op: training operation
        global_step: scalar.
        summaries: training summary node
        '''
        # forward
        memory, sents1, src_masks = self.encode(xs)
        logits, preds, y, sents2 = self.decode(ys, memory, src_masks)

        # train scheme
        y_ = label_smoothing(tf.one_hot(y, depth=self.hp.vocab_size))
        ce = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_)
        nonpadding = tf.to_float(tf.not_equal(y, self.token2idx["<pad>"]))  # 0: <pad>
        loss = tf.reduce_sum(ce * nonpadding) / (tf.reduce_sum(nonpadding) + 1e-7)

        global_step = tf.train.get_or_create_global_step()
        lr = noam_scheme(self.hp.lr, global_step, self.hp.warmup_steps)
        optimizer = tf.train.AdamOptimizer(lr)
        train_op = optimizer.minimize(loss, global_step=global_step)

        tf.summary.scalar('lr', lr)
        tf.summary.scalar("loss", loss)
        tf.summary.scalar("global_step", global_step)

        summaries = tf.summary.merge_all()

        return loss, train_op, global_step, summaries

    def eval(self, xs, ys):
        '''Predicts autoregressively
        At inference, input ys is ignored.
        Returns
        y_hat: (N, T2)
        '''
        decoder_inputs, y, y_seqlen, sents2 = ys

        decoder_inputs = tf.ones((tf.shape(xs[0])[0], 1), tf.int32) * self.token2idx["<s>"]
        ys = (decoder_inputs, y, y_seqlen, sents2)

        memory, sents1, src_masks = self.encode(xs, False)

        for _ in tqdm(range(self.hp.maxlen2)):
            logits, y_hat, y, sents2 = self.decode(ys, memory, src_masks, False)
            if tf.reduce_sum(y_hat, 1) == self.token2idx["<pad>"]: break

            _decoder_inputs = tf.concat((decoder_inputs, y_hat), 1)
            ys = (_decoder_inputs, y, y_seqlen, sents2)

        # monitor a random sample
        n = tf.random_uniform((), 0, tf.shape(y_hat)[0]-1, tf.int32)
        sent1 = sents1[n]
        pred = convert_idx_to_token_tensor(y_hat[n], self.idx2token)
        sent2 = sents2[n]

        tf.summary.text("sent1", sent1)
        tf.summary.text("pred", pred)
        tf.summary.text("sent2", sent2)
        summaries = tf.summary.merge_all()

        return y_hat, summaries

# Train and Eval

In [29]:
hparams = Hparams()
parser = hparams.parser
hp = parser.parse_args()
# hparam define --begin
hp.logdir='./transformer_checkpoint'
hp.evaldir='./eval'
hp.testdir='./test'
hp.vocab_size=1000
hp.batch_size=32
hp.eval_batch_size=32
hp.lr=0.0003
hp.warmup_steps=2000
hp.num_epochs=10
hp.d_model=512
hp.d_ff=512
hp.num_blocks=4
hp.num_heads=4
hp.maxlen1=100
hp.maxlen2=100
hp.dropout_rate=0.3
hp.smoothing=0.1
# hparam define --end
save_hparams(hp, hp.logdir)
tf.reset_default_graph()

train_batches, num_train_batches, num_train_samples = get_batch(hp.train1, hp.train2,
                                             hp.maxlen1, hp.maxlen2,
                                             hp.vocab, hp.batch_size,
                                             shuffle=True)
eval_batches, num_eval_batches, num_eval_samples = get_batch(hp.eval1, hp.eval2,
                                             100000, 100000,
                                             hp.vocab, hp.batch_size,
                                             shuffle=False)

In [30]:
iter = tf.data.Iterator.from_structure(train_batches.output_types, train_batches.output_shapes)
xs, ys = iter.get_next()

In [31]:
train_init_op = iter.make_initializer(train_batches)
eval_init_op = iter.make_initializer(eval_batches)

In [32]:
print(hp)
m = Transformer(hp)
loss, train_op, global_step, train_summaries = m.train(xs, ys)
y_hat, eval_summaries = m.eval(xs, ys)

Namespace(batch_size=32, ckpt='./transformer_checkpoints', d_ff=512, d_model=512, dropout_rate=0.3, eval1='./iwslt2016/segmented/eval.de.bpe', eval2='./iwslt2016/segmented/eval.en.bpe', eval3='./iwslt2016/prepro/eval.en', eval_batch_size=32, evaldir='./eval', f='/home/duclv/.local/share/jupyter/runtime/kernel-5beeed36-9773-48f7-ae30-3a5547a9f6ea.json', logdir='./transformer_checkpoint', lr=0.0003, maxlen1=100, maxlen2=100, num_blocks=4, num_epochs=10, num_heads=4, smoothing=0.1, test1='./iwslt2016/segmented/test.de.bpe', test2='./iwslt2016/prepro/test.en', test_batch_size=128, testdir='./test', train1='./iwslt2016/segmented/train.de.bpe', train2='./iwslt2016/segmented/train.en.bpe', vocab='./iwslt2016/segmented/bpe.vocab', vocab_size=1000, warmup_steps=2000)


100%|██████████| 100/100 [03:17<00:00,  1.98s/it]


In [19]:
saver = tf.train.Saver(max_to_keep=hp.num_epochs)
with tf.Session() as sess:
    ckpt = tf.train.latest_checkpoint(hp.logdir)
    if ckpt is None:
        sess.run(tf.global_variables_initializer())
        save_variable_specs(os.path.join(hp.logdir, "specs"))
    else:
        saver.restore(sess, ckpt)

    summary_writer = tf.summary.FileWriter(hp.logdir, sess.graph)

    sess.run(train_init_op)
    total_steps = hp.num_epochs * num_train_batches
    _gs = sess.run(global_step)
    for i in tqdm(range(_gs, total_steps+1)):
        _, _gs, _summary = sess.run([train_op, global_step, train_summaries])
        epoch = math.ceil(_gs / num_train_batches)
        summary_writer.add_summary(_summary, _gs)

        if _gs and _gs % num_train_batches == 0:
            _loss = sess.run(loss) # train loss

            _, _eval_summaries = sess.run([eval_init_op, eval_summaries])
            summary_writer.add_summary(_eval_summaries, _gs)

            hypotheses = get_hypotheses(num_eval_batches, num_eval_samples, sess, y_hat, m.idx2token)

            model_output = "iwslt2016_E%02dL%.2f" % (epoch, _loss)
            if not os.path.exists(hp.evaldir): os.makedirs(hp.evaldir)
            translation = os.path.join(hp.evaldir, model_output)
            with open(translation, 'w') as fout:
                fout.write("\n".join(hypotheses))

            calc_bleu(hp.eval3, translation)

            ckpt_name = os.path.join(hp.logdir, model_output)
            saver.save(sess, ckpt_name, global_step=_gs)

            sess.run(train_init_op)
    summary_writer.close()


INFO:root:Variables info has been saved.


num_params:  21241347


 10%|▉         | 2966/29661 [09:49<7:34:39,  1.02s/it]

bleu_score_report:  BLEU = 5.96, 31.0/10.1/3.5/1.2 (BP=0.981, ration=0.981)



 20%|█▉        | 5932/29661 [20:49<1:07:34,  5.85it/s]  

...

 90%|████████▉ | 26694/29661 [1:31:42<09:15,  5.34it/s]   

bleu_score_report:  BLEU = 13.74, 43.2/20.2/10.0/5.2 (BP=0.944, ration=0.946)



100%|█████████▉| 29660/29661 [1:41:48<00:00,  5.05it/s]   

bleu_score_report:  BLEU = 13.69, 41.9/19.4/9.5/4.9 (BP=0.983, ration=0.983)



100%|██████████| 29661/29661 [1:42:33<00:00,  4.82it/s]


# Test

In [33]:
hp.ckpt = './transformer_checkpoint'
load_hparams(hp, hp.ckpt)

test_batches, num_test_batches, num_test_samples  = get_batch(hp.test1, hp.test1,
                                              100000, 100000,
                                              hp.vocab, hp.test_batch_size,
                                              shuffle=False)
iter = tf.data.Iterator.from_structure(test_batches.output_types, test_batches.output_shapes)
xs, ys = iter.get_next()

test_init_op = iter.make_initializer(test_batches)

In [34]:
y_hat, _ = m.eval(xs, ys)

100%|██████████| 100/100 [01:52<00:00,  1.13s/it]


In [37]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
with tf.Session() as sess:
    ckpt_ = tf.train.latest_checkpoint(hp.ckpt)
    ckpt = hp.ckpt if ckpt_ is None else ckpt_ # None: ckpt is a file. otherwise dir.
    saver = tf.train.Saver()

    saver.restore(sess, ckpt)

    sess.run(test_init_op)

    hypotheses = get_hypotheses(num_test_batches, num_test_samples, sess, y_hat, m.idx2token)

    model_output = ckpt.split("/")[-1]
    if not os.path.exists(hp.testdir): os.makedirs(hp.testdir)
    translation = os.path.join(hp.testdir, model_output)
    with open(translation, 'w') as fout:
        fout.write("\n".join(hypotheses))

    calc_bleu(hp.test2, translation)

INFO:tensorflow:Restoring parameters from ./transformer_checkpoint/iwslt2016_E10L2.49-59320


INFO:tensorflow:Restoring parameters from ./transformer_checkpoint/iwslt2016_E10L2.49-59320


ResourceExhaustedError: OOM when allocating tensor with shape[128,138,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node encoder_2/embedding_lookup (defined at <ipython-input-12-5ce68a37eb0f>:31)  = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@decoder_200/embedding_lookup"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](shared_weight_matrix/concat, IteratorGetNext_1/_883, encoder_2/embedding_lookup/axis)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node decoder_137/num_blocks_1/positionwise_feedforward/dense_1/Tensordot/Shape/_8337}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_43973...rdot/Shape", tensor_type=DT_INT32, _device="/jo